<a href="https://colab.research.google.com/github/Ddshsp/lab/blob/main/%D0%A8%D0%94%D0%94_%D0%9B%D0%A01_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from itertools import accumulate
from tqdm import tqdm

# Генерация данных

In [ ]:
def func(n: int) -> pd.DataFrame:
    data = {
        "gender": np.random.choice(["Male", "Female"], n),
        "age": np.random.randint(18, 65, n),
        "income": np.random.randint(20000, 150000, n),
        "profession": np.random.choice(["Engineer", "Doctor", "Teacher", "Lawyer"], n),
    }
    return pd.DataFrame(data)

In [ ]:
n = 1_000_000
df = func(n)

In [ ]:
df.describe()

,age,income
count,1000000.000000,1000000.000000
mean,40.992704,85082.694896
std,13.571183,37520.324990
min,18.000000,20000.000000
25%,29.000000,52574.000000
50%,41.000000,85185.000000
75%,53.000000,117544.000000
max,64.000000,149999.000000


In [ ]:
df.head(5)

,gender,age,income,profession
0,Female,63,98858,Engineer
1,Female,62,137632,Teacher
2,Male,50,67315,Lawyer
3,Female,54,32917,Teacher
4,Female,44,96156,Teacher


# Замерим объем памяти

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   gender      1000000 non-null  object
 1   age         1000000 non-null  int64 
 2   income      1000000 non-null  int64 
 3   profession  1000000 non-null  object
dtypes: int64(2), object(2)
memory usage: 30.5+ MB


In [ ]:
df.memory_usage()

,0
Index,128
gender,8000000
age,8000000
income,8000000
profession,8000000


# Расчет времени выполнения

In [ ]:
%%timeit
df.to_csv("check")

2.63 s ± 58.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
df = pd.read_csv("check")

513 ms ± 9.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
df["age"].mean()

1.88 ms ± 556 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
df["income"].max()

457 µs ± 143 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
df["gender"].mode()[0]

78.1 ms ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
df['profession'].value_counts().idxmin()

95.4 ms ± 30.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Itertools

In [ ]:
df = func(10_000_000)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   gender      object
 1   age         int64 
 2   income      int64 
 3   profession  object
dtypes: int64(2), object(2)
memory usage: 305.2+ MB


In [ ]:
df.memory_usage()

,0
Index,128
gender,80000000
age,80000000
income,80000000
profession,80000000


In [ ]:
%%timeit
df.to_csv("check")

23.2 s ± 1.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
df = pd.read_csv("check")

6.3 s ± 679 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:

class Avg:
  __slots__ = ('n', 'sumix')
  def __init__(self, sumix):
    self.n, self.sumix = 1, sumix
  def __add__(self, other):
    self.n += other.n
    self.sumix += other.sumix
    return self
  def __iadd__(self,other):
    return self+other
  def calc(self):
    return self.sumix / self.n

In [ ]:
%%timeit
for i in tqdm(accumulate(map(lambda x:Avg(x),df['age'].values))):
  l = i
print(l.calc())


10000000it [00:13, 766246.64it/s]


40.9946846


10000000it [00:11, 870648.40it/s]


40.9946846


10000000it [00:11, 890267.38it/s]


...

40.9946846


10000000it [00:11, 889647.65it/s]


40.9946846


10000000it [00:09, 1014197.22it/s]

40.9946846
11.1 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
class Dictator:
  def __init__(self, val=None):
    if val is None:
      self.male, self.female = 0, 0
    if val == 'male':
      self.male = 1
      self.female = 0
    else:
      self.male = 0
      self.female = 1
  def __add__(self,other):
    self.male += other.male
    self.female += other.female
    return self
  def __iadd__(self,other):
    return self+other
  def calc(self):
    if self.male > self.female:
      return 'male'
    elif self.male == self.female:
      return 'equal'
    return 'female'

In [ ]:
%%timeit
for i in tqdm(accumulate(map(lambda x:Dictator(x),df['gender'].values))):
  l = i

print(l.calc())

10000000it [00:12, 798486.18it/s]


female


10000000it [00:13, 740447.79it/s]


female


10000000it [00:12, 785765.95it/s]


...

female


10000000it [00:12, 800352.73it/s]


female


10000000it [00:11, 839726.79it/s]

female
12.4 s ± 570 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
class Proff:
  def __init__(self, val=None):
    self.diction = dict(zip(["Engineer", "Doctor", "Teacher", "Lawyer"],[0]*4))
    if val is not None:
      self.diction[val] += 1
  def __add__(self,other):
    for i in self.diction.keys():
      self.diction[i] += other.diction[i]
    return self
  def __iadd__(self,other):
    return self+other
  def calc(self):
    return min(self.diction.keys(), key = lambda x: self.diction[x])

In [ ]:
%%timeit
for i in tqdm(accumulate(map(lambda x:Proff(x),df['profession'].values))):
  l = i
l.calc()

10000000it [00:31, 320594.12it/s]
10000000it [00:30, 331279.46it/s]
10000000it [00:32, 310976.69it/s]
10000000it [00:30, 322805.84it/s]
10000000it [00:29, 341425.32it/s]
10000000it [00:28, 352124.42it/s]
10000000it [00:27, 360111.86it/s]
10000000it [00:26, 371294.62it/s]

29.4 s ± 1.71 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
import threading
def func(link, iter):
  l = None
  for i in iter:
    l = i

  print(link, l.calc())
link_list = ("Средний возраст:","Средняя зарплата:","Самый частый пол:","Самая редкая профессия:")
iter_list = (accumulate(map(lambda x:Avg(x),df['age'].values)),accumulate(map(lambda x:Avg(x),df['income'].values)),accumulate(map(lambda x:Dictator(x),df['gender'].values)),accumulate(map(lambda x:Proff(x),df['profession'].values)))
threads = []

for link, iter in zip(link_list, iter_list):
    my_thread = threading.Thread(target=func, args=(link, iter))
    threads.append(my_thread)
    print(f'Thread {link}')
    my_thread.start()

for thread in tqdm(threads):
    thread.join()  # Ждем завершения всех потоков

Thread Средний возраст:
Thread Средняя зарплата:
Thread Самый частый пол:
Thread Самая редкая профессия:


 25%|██▌       | 1/4 [00:34<01:44, 34.99s/it]

Средний возраст: 40.9946846


 50%|█████     | 2/4 [00:38<00:32, 16.47s/it]

Средняя зарплата: 85012.7815098


 75%|███████▌  | 3/4 [00:39<00:09,  9.62s/it]

Самый частый пол: female


100%|██████████| 4/4 [00:58<00:00, 14.62s/it]

Самая редкая профессия: Engineer


In [ ]:
import csv
import numpy as np

num_rows = 10000000

# Создание файла CSV и запись данных
with open('check', 'w', newline='') as csvfile:
    fieldnames = ['gender', 'age', 'profession', 'income']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Записываем заголовок
    writer.writeheader()

    # Записываем данные
    for i in tqdm(range(num_rows)):
        sex_values = np.random.choice(['male', 'female'])
        age_values = np.random.randint(18, 65)
        profession_values = np.random.choice(["Engineer", "Doctor", "Teacher", "Lawyer"])
        income_values = np.random.normal(50000, 250000)
        writer.writerow({'gender': sex_values, 'age': age_values, 'profession': profession_values, 'income': income_values})

# Загрузка данных из файла CSV



100%|██████████| 10000000/10000000 [09:28<00:00, 17592.61it/s]


In [ ]:
batch_size = 1000000
proc_size = batch_size//4
import csv
import threading

class Avg_new(Avg):
  def __init__(self,n,sumix):
    self.n = n
    self.sumix = sumix


def function(it, array):
    avg_age = Avg_new(0, 0)
    avg_incom = Avg_new(0, 0)
    pop_sex = Dictator()
    prof = Proff()
    for i in array:
      avg_age += Avg_new(1,int(i['age']))
      avg_incom += Avg_new(1, float(i['income']))
      pop_sex += Dictator(i['gender'])
      prof += Proff(i['profession'])
    result[it] = (avg_age, avg_incom, pop_sex, prof)

with open('check', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    avg_age_final = Avg_new(0, 0)
    avg_incom_final = Avg_new(0, 0)
    pop_sex_final = Dictator()
    prof_final = Proff()

    for batch_num in range(10):
      threads = []
      arr = []
      i = 0
      result = [0]*4

      for _ in range(batch_size):
        arr.append(next(reader))
        if len(arr) == proc_size:
          my_thread = threading.Thread(target=function,args=(i,arr.copy()))
          arr = []
          i += 1
          threads.append(my_thread)
          my_thread.start()

      for thread in threads:
        thread.join()  # Ждем завершения всех потоков
      for i in result:
        avg_age_final += i[0]
        avg_incom_final += i[1]
        pop_sex_final += i[2]
        prof_final += i[3]
    print(avg_age_final.calc())
    print(avg_incom_final.calc())
    print(pop_sex_final.calc())
    print(prof_final.calc())

        # threads = []



        # for link, iter in zip(link_list, iter_list):
        #     my_thread = threading.Thread(target=func, args=(link, iter))
        #     threads.append(my_thread)
        #     print(f'Thread {link}')
        #     my_thread.start()

40.997026
49828.14475609023
female
Teacher


In [ ]:
import csv
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import threading
import concurrent

locker = threading.Lock()
WORKER = 4

def generate_data(di: dict) -> list:
  keys = di.keys()
  return [dict(zip(keys,i)) for i in zip(*di.values())]

def func(n:int):
  def wrap() -> None:
    data = {
        "gender": np.random.choice(["Male", "Female"], n),
        "age": np.random.randint(18, 65, n),
        "income": np.random.randint(20000, 150000, n),
        "profession": np.random.choice(["Engineer", "Doctor", "Teacher", "Lawyer"], n),
    }
    data = generate_data(data)
    with locker:
      writer.writerows(data)
  return wrap


num_rows = 400_000_000
batch_size = 1_000_000
# Создание файла CSV и запись данных
with open('check', 'w', newline='') as csvfile:
    fieldnames = ['gender', 'age', 'profession', 'income']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Записываем заголовок
    writer.writeheader()

    # Записываем данные
    with ThreadPoolExecutor(max_workers=WORKER) as executor:
      futures = {executor.submit(func(batch_size)): i for i in range(int(num_rows/batch_size))}
    for future in tqdm(concurrent.futures.as_completed(futures)):
      image_data = future.result()



# Загрузка данных из файла CSV



100%|██████████| 400/400 [49:33<00:00,  7.43s/it]


In [ ]:
!cat check|wc -l

400000001


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import operator
import csv
import threading
from tqdm import tqdm
import queue
import time

def read(reader: csv.DictReader, q: queue.Queue, e: threading.Event):
    for i in tqdm(reader):
        q.put(i)
    e.set()
    q.join()

def func():
    # avg
    co = 0
    su = 0
    # max
    ma = 0
    # gender
    ge = {}
    # prof
    pr = {}

    locker = threading.Lock()

    def process(di: queue.Queue, event: threading.Event):
        nonlocal co, su, ma, ge, pr, locker
        # avg
        co2 = 0
        su2 = 0
        # max
        ma2 = 0
        # gender
        ge2 = {}
        # prof
        pr2 = {}
        while not (event.is_set() and di.empty()):
            try:
              i = di.get(block=False)
            except Exception:
              continue
            di.task_done()
            co2 += 1
            su2 += int(i["age"])
            ma2 = max(ma2, int(i["income"]))
            ge2[i["gender"]] = ge2.get(i["gender"], 0) + 1
            pr2[i["profession"]] = pr2.get(i["profession"], 0) + 1
        with locker:
            co += co2
            su += su2
            ma = max(ma, ma2)
            for key, value in ge2.items():
                ge[key] = ge.get(key, 0) + value
            for key, value in pr2.items():
                pr[key] = pr.get(key, 0) + value

    def ans():
        ans1 = su / co
        ans2 = ma
        ans3 = max(ge.items(), key=operator.itemgetter(1))[0]
        ans4 = min(pr.items(), key=operator.itemgetter(1))[0]
        return ans1, ans2, ans3, ans4

    return process, ans

process, ans = func()
WORKER = 4
batch_size = 10_000_000

with open('check2', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    q = queue.Queue(maxsize=WORKER * batch_size)
    event = threading.Event()

    with ThreadPoolExecutor(max_workers=WORKER) as executor:
        # reader_thread = executor.submit(read, reader, q, event)

        for _ in range(WORKER):
            executor.submit(process, q, event)

        # reader_thread.result()
        read(reader,q,event)

        # q.join()

print(ans())


12018754it [14:50, 13499.08it/s]


KeyboardInterrupt: 

In [ ]:
!pip install polars

In [ ]:
import polars as pl
from concurrent.futures import ThreadPoolExecutor
import threading
from tqdm import tqdm

locker = threading.Lock()
reader = pl.read_csv_batched("check2",batch_size=1_000_000)

# avg
co = 0
su = 0
# max
ma = 0
# gender
ge = {}
# prof
pr = {}


def process(df: pl.DataFrame):
  global co, su, ma, ge, pr
  co2 = df.select(pl.count("age")).item()
  su2 = df.select(pl.sum("age")).item()
  ma2 = df.select(pl.max("income")).item()
  temp2 = df.group_by("gender").count()
  keys = tuple(temp2["gender"])
  ge2 = dict([(key, temp2.select(pl.col("count").where(pl.col("gender")==key)).item()) for key in keys])
  temp2 = df.group_by("profession").count()
  keys = tuple(temp2["profession"])
  pr2 = dict([(key, temp2.select(pl.col("count").where(pl.col("profession")==key)).item()) for key in keys])

  with locker:
      co += co2
      su += su2
      ma = max(ma, ma2)
      for key, value in ge2.items():
          ge[key] = ge.get(key, 0) + value
      for key, value in pr2.items():
          pr[key] = pr.get(key, 0) + value


for i in tqdm(range(400)):
  batches = reader.next_batches(4)
  with ThreadPoolExecutor(max_workers=4) as executor:
    try:
      executor.map(process, batches)
    except Exception:
      continue



100%|██████████| 400/400 [02:14<00:00,  2.97it/s]


In [ ]:
su/co

40.9999932875

In [ ]:
ma

149999

In [ ]:
ge

{'Female': 199998870, 'Male': 200001130}

In [ ]:
pr

{'Doctor': 99978202,
 'Engineer': 99996099,
 'Lawyer': 100008910,
 'Teacher': 100016789}